In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from ebany_research.llm_lora.changed_mistral import (
    LinearLora,
    ChangedMistralForCausalLM,
)
from transformers import AutoTokenizer, AutoConfig
import torch

model_name = "Open-Orca/Mistral-7B-OpenOrca"
lora_model_name = "ebany_research/llm_lora/models/"
lora_model_name += "52[6c_11c_14_17c_20c_22c_25c_26][6_10_11_14_17_20_22_25_26_30][6c_10c_11c_14c_17c_20_22c_25c_26c_30c]"

config = AutoConfig.from_pretrained(lora_model_name)
student_model = ChangedMistralForCausalLM.from_pretrained(
    lora_model_name,
    device_map={"": 0},
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
)
student_model = student_model.eval()
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:21<00:00,  7.20s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
with torch.no_grad():
    chat = [
        {
            "role": "system",
            "content": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer.",
        },
        # {
        #     "role": "user",
        #     "content": """Please add punctuation to this: The cause of the explosion has never been clear the affected area was completely uninhabited""",
        # },
        # {"role": "assistant", "content": dataset_item["response"]},
    ]
    inputs_text = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        # add_generation_prompt=True,
    )
    # print(inputs)
    inputs = tokenizer(
        inputs_text,
        return_tensors="pt",
        truncation=True,
        # max_length=4096,
        max_length=2048,
    ).to(student_model.device)

    outputs = student_model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        # top_k=50,
        # top_p=0.95,
    )
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=False)
    print(outputs[0])

In [3]:
tokenizer(
    [
        "hello world hello world hello worldhello world",
        "hello",
    ],
    # return_tensors="pt",
    truncation=True,
    # max_length=4096,
    max_length=2048,
    padding=True,
)

{'input_ids': [[1, 6312, 28709, 1526, 6312, 28709, 1526, 6312, 28709, 1526, 21558, 1526], [32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 1, 6312, 28709]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]]}

In [11]:
chat = [
    {
        "role": "system",
        "content": "You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.",
    },
    {
        "role": "user",
        "content": """David has three sisters. Each of them has one brother. How many brothers does David have?""",
    },
    # {"role": "assistant", "content": "The Confederation of British Industry (CBI) and British Chambers of Commerce have called for market stability and political clarity following the UK's decision to leave the EU. Bank of England Governor Mark Carney has promised support for financial markets, announcing the Bank's preparedness to provide £250bn to support markets during any periods of instability. Business leaders have expressed concerns over resulting uncertainty slowing economic activity, and companies are seeking immediate clarity on a timetable to leave the EU. Meanwhile, union leaders' priority is to protect jobs and living standards of the UK's working population."},
]
inputs = tokenizer.apply_chat_template(
    chat,
    tokenize=False,
    # add_generation_prompt=True,
)
inputs

'<|im_start|>system\nYou are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.<|im_end|>\n<|im_start|>user\nDavid has three sisters. Each of them has one brother. How many brothers does David have?<|im_end|>\n'

### mask user inputs

In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset

model_name = "Open-Orca/Mistral-7B-OpenOrca"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

dataset = load_dataset("dim/openaccess-ai-collective-oo-gpt4-filtered")
dataset = dataset["train"].to_list()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/user-name-goes-here/.local/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [4]:
chat = [
    {
        "role": "system",
        "content": dataset[0]["system_prompt"],
    },
    {
        "role": "user",
        "content": dataset[0]["question"],
    },
    # {"role": "assistant", "content": "The Confederation of British Industry (CBI) and British Chambers of Commerce have called for market stability and political clarity following the UK's decision to leave the EU. Bank of England Governor Mark Carney has promised support for financial markets, announcing the Bank's preparedness to provide £250bn to support markets during any periods of instability. Business leaders have expressed concerns over resulting uncertainty slowing economic activity, and companies are seeking immediate clarity on a timetable to leave the EU. Meanwhile, union leaders' priority is to protect jobs and living standards of the UK's working population."},
]
user_prompt = tokenizer.apply_chat_template(
    chat,
    tokenize=False,
    # add_generation_prompt=True,
)
result = tokenizer(
    user_prompt,
    truncation=True,
    max_length=2048,
    padding=False,
    return_tensors=None,
)
result

{'input_ids': [1, 32001, 1587, 13, 1976, 460, 264, 10865, 13892, 28725, 693, 1743, 3084, 13268, 28723, 15403, 737, 368, 460, 24402, 298, 264, 3359, 879, 1571, 28723, 32000, 32001, 2188, 13, 28790, 28706, 1321, 1723, 28862, 21805, 1101, 3850, 28719, 28820, 668, 1254, 28862, 8269, 427, 363, 28873, 17957, 19506, 28871, 3589, 28723, 13, 13, 3131, 10020, 298, 4300, 13, 13, 27871, 28747, 32000, 28705, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
dataset[0]

{'id': 'flan.2389905',
 'system_prompt': 'You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.',
 'question': 'Ve skutečnosti... ...má společné se všemi loděmi.\n\nTranslate to English\n\nEnglish:',
 'response': 'In reality... ...has in common with all the ships.\n\nEnglish: In reality, it shares something with all the ships.',
 '__index_level_0__': 923823}

8275823265479259487

In [29]:
import abc
from transformers import BatchEncoding, PreTrainedTokenizer
from typing import Dict, List, Tuple, Union


class OpenOrcaSystemDataPrompter:
    """
    Alpaca Style Prompter that uses system prompts from the dataset, with OpenOrca prompts
    """

    def get_prompt(
        self,
        instruction="",
        system="",
    ):
        self.instruction_prompt = (
            f"<|im_start|>user\n{instruction}<|im_end|>\n<|im_start|>assistant\n"
        )
        self.system_prompt = f"<|im_start|>system\n{system}<|im_end|>\n"
        return self.system_prompt + self.instruction_prompt


class PromptTokenizingStrategy(abc.ABC):
    """
    Abstract class for tokenizing strategies
    """

    def __init__(
        self,
        prompter=None,
        tokenizer=None,
        train_on_inputs: bool = False,
        sequence_len: int = 2048,
    ):
        self.prompter = prompter
        self.tokenizer: PreTrainedTokenizer = tokenizer
        self.train_on_inputs = train_on_inputs
        # sequence_len and max_length can be different for CompletionPromptTokenizingStrategy.
        # TODO: Document how they are different.
        self.sequence_len = sequence_len
        self.max_length = sequence_len

        self.prompter = OpenOrcaSystemDataPrompter()

    @abc.abstractmethod
    def tokenize_prompt(self, prompt):
        pass

    @property
    def supports_batched(self):
        return False

    def _tokenize(
        self, prompt: str, add_eos_token: bool = True, strip_bos_token: bool = False
    ) -> BatchEncoding:
        empty = BatchEncoding(data={"input_ids": [], "attention_mask": []})
        if not prompt:
            print("Empty text requested for tokenization.")
            return empty

        result = self.tokenizer(
            prompt,
            truncation=True,
            max_length=self.max_length,
            padding=False,
            return_tensors=None,
        )
        if len(result["input_ids"]) == 0:
            print("Tokenizer result is empty. You may want to audit your dataset")
            return empty

        if (
            result["input_ids"][-1] != self.tokenizer.eos_token_id
            and len(result["input_ids"]) < self.max_length
            and add_eos_token
        ):
            result["input_ids"].append(self.tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        if result["input_ids"][0] == self.tokenizer.bos_token_id and strip_bos_token:
            result["input_ids"] = result["input_ids"][1:]
            result["attention_mask"] = result["attention_mask"][1:]

        result["labels"] = result["input_ids"].copy()
        return result


class InstructionPromptTokenizingStrategy(PromptTokenizingStrategy):
    """
    Tokenizing strategy for instruction-based prompts.
    """

    def parse_instruction_fields(
        self, prompt
    ) -> Union[Tuple[str, str, str], Tuple[str, str, str, str]]:
        return (prompt["system_prompt"], prompt["question"], prompt["response"])

    def tokenize_prompt(self, prompt):
        (
            instruction,
            input,  # pylint: disable=redefined-builtin
            response,
        ) = self.parse_instruction_fields(prompt)

        user_prompt = self.prompter.get_prompt(
            instruction=input,
            system=instruction,
        )
        tokenized_prompt = self._tokenize(user_prompt, add_eos_token=False)
        if not self.train_on_inputs:
            user_prompt_len = len(tokenized_prompt["input_ids"])
            # TODO this could be sped up using numpy array slicing
            tokenized_prompt["labels"] = [-100] * user_prompt_len

        tokenized_res_prompt = self._tokenize(
            response, strip_bos_token=True, add_eos_token=True
        )
        tokenized_prompt["input_ids"] += tokenized_res_prompt["input_ids"]
        tokenized_prompt["attention_mask"] += tokenized_res_prompt["attention_mask"]
        tokenized_prompt["labels"] += tokenized_res_prompt["input_ids"]

        return tokenized_prompt

example = dataset[10]
prompt_tokenizer = InstructionPromptTokenizingStrategy(tokenizer=tokenizer)
result = prompt_tokenizer.tokenize_prompt(example)

print(tokenizer.decode(result["input_ids"]))

import torch

labels = torch.tensor(result["labels"])
labels = labels[(labels == -100).sum():] 
print('---')
print(tokenizer.decode(labels))
print('---')
print(example['response'])

<s><|im_start|> system
You are an AI assistant. You will be given a task. You must generate a detailed and long answer.<|im_end|><|im_start|> user
Produce a long descriptive sentence that uses all these words: Bhajji, country, India; India, demonym, Indian people; India, leader, Narendra Modi; India, leader, Sumitra Mahajan; Bhajji, region, Karnataka<|im_end|><|im_start|> assistant
 In the culturally diverse country of India, where Bhajji, a popular deep-fried snack, originated within the vibrant region of Karnataka, known for its piquant spices and culinary expertise, the Indian people, denoted by the unique demonym, form a strong and passionate community that treasures its vastly distinctive values, traditions, and identity, underneath the political leadership of prominent figures such as the inspiring Prime Minister Narendra Modi and the experienced, diligent Sumitra Mahajan, who have both contributed to reshaping the Indian sociopolitical landscape while endeavoring to ensure a bri

In reality... ...has in common with all the ships.

English: In reality, it shares something with all the ships.<|im_end|>


<function print>

In [21]:
tokenizer('')

{'input_ids': [1], 'attention_mask': [1]}